# HLS Data Analysis Notebook
## Index
[Fetching Data from Archiver](#fetch-data)
-> [Input section](#fetch-data-input)

[Calculate FFT for the loaded PV's data](#calc-FFT)

<a id='fetch-data'></a>
## Fetching data from Archiver

In [2]:
%gui asyncio
from IPython.display import display
import ipywidgets as w
import ipydatetime

import asyncio
import aiohttp

import pandas as pd

from datetime import datetime

In [3]:
ARCHIVE_URL = 'http://10.0.38.42/retrieval/data/getData.json'
data = pd.DataFrame(data={'datetime': [], 'pv': []})

In [4]:
async def fetch(session, pv, time_from, time_to, is_optimized, mean_minutes):
    if is_optimized:
        pv_query = f'mean_{int(60*mean_minutes)}({pv})'
    else:
        pv_query = pv
    query = {'pv': pv_query, 'from': time_from, 'to': time_to}
    response_as_json = {}
    async with session.get(ARCHIVE_URL, params=query) as response:
        try:
            response_as_json = await response.json()
        except aiohttp.client_exceptions.ContentTypeError:
            print(f'Failed to retrieve data from PV {pv}.')
            response_as_json = None
        return response_as_json

async def retrieve_data(pvs, time_from, time_to, isOptimized=False, mean_minutes=0):
    async with aiohttp.ClientSession() as session:
        data = await asyncio.gather(*[fetch(session, pv, time_from, time_to, isOptimized, mean_minutes) for pv in pvs])
        return data

async def fetch_data(pv_list, timespam):
    global data
    optimize = True
    mean_minutes = 3
    res = await retrieve_data(pv_list, timespam['init'], timespam['end'], optimize, mean_minutes)
    try:
        # cleaning response
        res_mapped = list(map(lambda x: x[0]['data'], res))
    except TypeError:
        log.append_stdout('Incorrect PV(s) name(s) or bad timespam. Fetching failed.\n')
        return
    values = [list(map(lambda x: x['val'], pv_data)) for pv_data in res_mapped]
    ts = [list(map(lambda x: datetime.fromtimestamp(x['secs']).strftime("%d.%m.%y %H:%M"), pv_data)) for pv_data in res_mapped]
    # creating pandas dataframe object
    d = {'datetime': ts[0]}
    for val, pv in zip(values, pv_list):
        d[pv] = val
    data = pd.DataFrame(data=d)
    # indexing by datetime
    data.reset_index(drop=True, inplace=True)
    data = data.set_index('datetime')
    
    log.append_stdout('Fetched!\n')
    with pd.option_context('display.max_rows',6):
        with pd.option_context('display.max_columns',8):
            loaded_data.append_display_data(data)
    dropdown_pv.options = data.columns.values
    return data

def generate_all_sensors_text():
    sectors = [17, 16, 15, 14, 13, 1, 1, 20, 19, 18, 6, 6, 5, 4, 3, 11, 11, 10, 9, 8]
    axis = [4, 1, 59, 57, 54, 18, 16, 14, 12, 9, 33, 31, 29, 27, 24, 48, 46, 44, 42, 39]
    quadrant = ['NE5', 'NE4', 'NE3', 'NE2', 'NE1', 'SE5', 'SE4', 'SE3', 'SE2', 'SE1', 'SW5', 'SW4', 'SW3', 'SW2', 'SW1', 'NW5', 'NW4', 'NW3', 'NW2', 'NW1']
    pvs_text = ''
    for sector, ax, quad in zip(sectors, axis, quadrant):
        pvs_text += f'TU-{sector:02d}C:SS-HLS-Ax{ax:02d}{quad}:Level-Mon\n'
    pvs_text = pvs_text[:-1]
    return pvs_text

def call_fetch(b):
    try:
        dt_init = datetime_init_picker.value
        dt_init = (dt_init - dt_init.utcoffset()).replace(tzinfo=None).isoformat(timespec='milliseconds')+'Z'
        dt_end = datetime_end_picker.value
        dt_end = (dt_end - dt_end.utcoffset()).replace(tzinfo=None).isoformat(timespec='milliseconds')+'Z'
        timespam = {"init": dt_init, "end": dt_end}
    except AttributeError:
        log.append_stdout('Please fill all the fields.\n')
        return
    loaded_data.clear_output()
    log.append_stdout('Fetching data...\n')
    
    pv_list = pv_input.value.split('\n')
    asyncio.create_task(fetch_data(pv_list, timespam))
    
def dropdown_predefined_pvs_handler(change):
    pv_set = change.new
    if (pv_set == 'All HLS sensors'):
        hls_pvs_text = generate_all_sensors_text()
        pv_input.value = hls_pvs_text
    elif (pv_set == 'RF Freq'):
        pv_input.value = 'RF-Gen:GeneralFreq-RB'
    else:
        pv_input.value = ''


In [5]:
# Defining widgets

dropdown_predefined_pvs = w.Dropdown(options=['None','All HLS sensors', 'RF Freq'], description='Predefined:')

pv_input = w.Textarea(
    placeholder='PVs list, separeted by Enter',
    disabled=False
)

log = w.Output(layout={'border': '1px solid black', 'height': '200px', 'width': '300px'})

loaded_data = w.Output()
with loaded_data:
    display(data)


fetch_btn = w.Button(
    description='Fetch data',
    disabled=False,
    button_style='success', # 'success', 'info', 'warning', 'danger' or ''
    icon='check'
)

datetime_init_picker = ipydatetime.DatetimePicker()
datetime_end_picker = ipydatetime.DatetimePicker()

# Defining event calls
fetch_btn.on_click(call_fetch)

dropdown_predefined_pvs.observe(dropdown_predefined_pvs_handler, names='value')

# Defining layout
column_left = w.VBox([w.Label(value='PVs:'), pv_input, dropdown_predefined_pvs, w.Label(value='Initial datetime:'), datetime_init_picker, w.Label('End datetime:'), datetime_end_picker, fetch_btn])
column_right = w.VBox([w.Label(value='Actions log'),log])

input_layout = w.HBox([column_left, column_right], layout={'display': 'flex', 'flex_flow': 'row', 'justify_content': 'space-around'})
output_layout = w.VBox([w.Label(value='Loaded data'), loaded_data], layout={'align_items': 'center'})

<a id='fetch-data-input'></a>
### Type in the name of the PV(s) and select the timespam to retrieve data

In [6]:
# Calling widgets
display(input_layout)
display(output_layout)

## Manipulate data (optional)

In [7]:
# creating widgets
dropdown_pv1 = w.Dropdown(options=data.columns.values)
dropdown_pv2 = w.Dropdown(options=data.columns.values)

# op = w.Dropdown(options=['Substract', 'Sum'])

op = w.ToggleButtons(
    options=['+', '-'],
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
#     icons=['check'] * 3
)

btn_apply = w.Button(
    description='Apply operation',
    disabled=False,
    button_style='success', # 'success', 'info', 'warning', 'danger' or ''
)

layout = w.HBox([w.VBox([w.Label(value='PV 1'), dropdown_pv1]), op, w.VBox([w.Label(value='PV 2'), dropdown_pv2])], layout={'align_items': 'center'})

display(layout)

<a id='calc-FFT'></a>
## Calculate FFT for the loaded PV's data and plot it

In [8]:
%matplotlib widget

import numpy as np
import time

import matplotlib.pylab as plt
import matplotlib.dates as mdates

from scipy.fft import fftfreq, rfft
from scipy.signal import butter, filtfilt
from scipy.signal import find_peaks, peak_widths

In [9]:
def calculate_fft():
    fft_plot_data = []
    filtered_timeseries = []
    
    timeserie = np.array(data.loc[:, dropdown_pv.value].values)
    timestamp = data.index.values[1]
    dt = data.index
    pv = dropdown_pv.value

    # ignoring datasets with more than 50% of repeated values (=no value)
    _, counts_elem = np.unique(timeserie, return_counts=True)
    repeated_elem = 0
    for count in counts_elem:
        if (count > 1):
            repeated_elem += count - 1
    percentage_repeated = repeated_elem/len(timeserie)
    if (percentage_repeated > 0.5):
        print(f'FFT of PV {pv} failed: percentage of static data of [{percentage_repeated}].')
        return None
    # defining sampling properties
    ts1 = time.mktime(datetime.strptime(data.index.values[0], "%d.%m.%y %H:%M").timetuple())
    ts2 = time.mktime(datetime.strptime(data.index.values[1], "%d.%m.%y %H:%M").timetuple())
    acq_period = ts2 - ts1
    T = acq_period # in seconds
    N = len(timeserie)
    # creating generic time x axis data
    time_ax = np.linspace(0, T*N, N)
    # creating frequency x axis data
    W = fftfreq(N, T)[:N//2+1]
    # applying filter if needed
    if (False):
        b, a = butter(4, [5.55e-5, 2.77e-4], 'bandpass', fs=1/T) #bandpass from 1h to 5h
        try:
            timeserie = filtfilt(b,a, timeserie)
        except ValueError:
            print(f'Filter not applyed in ts {timestamp}')
    # calculating fft
    yr = rfft(timeserie)
    yr = np.abs(yr)**2
    # finding the peaks of fft and its properties
    yp = 2/(N/2) * yr
    peaks, _ = find_peaks(yp)
    # prominences = peak_prominences(yp, peaks)[0]
    widths = peak_widths(yp, peaks, rel_height=0.5)[0]
    xf = np.array(W)
    xp = 1/xf/60/60
    try:
        y_max_peak = max(yp[peaks])
    except ValueError:
        print(f'Not calculated fft for pv {pv} in {timestamp}')
        return None
    x_max_peak = np.where(yp == y_max_peak)[0][0]
    period_max_peak = xp[x_max_peak]
    idx_max_peak = np.where(peaks == x_max_peak)[0][0]
    width_max_peak = widths[idx_max_peak]
    # filtering curves by the means of the amplitude of max peak and its width
#     if (pv == 'RF-Gen:GeneralFreq-RB'):
#         if (y_max_peak > 5000 and y_max_peak < 300000 and width_max_peak < 4):
#             fft_plot_data = (W, yr, N, timestamp, period_max_peak)
#             filtered_timeseries = filtered_data
#     elif (pv == 'HLS:C4_S2_LEVEL'):
#         if (y_max_peak > 1e-3 and width_max_peak < 3):
#             fft_plot_data = (W, yr, N, timestamp, period_max_peak)
#             filtered_timeseries = filtered_data
#     else:
#         print(f"Doesn't know how to plot {pv}, skiping it...", 'danger')
#         return None
    
    output = {'fft_data': (xp, yp), 'time_data': (dt, timeserie)}
    return output

def plot_fft_static(b):
    fft_data = calculate_fft()
    if (fft_data):
        plot_container.clear_output()
        plt.cla()
        with plot_container:
            fig = plt.figure()
            axs = fig.subplots(2,1)
            xp = np.array(fft_data['fft_data'][0])
            yp = fft_data['fft_data'][1]
            dt = fft_data['time_data'][0]
            ts = fft_data['time_data'][1]
            axs[0].plot(dt, ts)
            axs[1].plot(xp, yp)        

            locator = mdates.AutoDateLocator(minticks=5, maxticks=10)
            formatter = mdates.ConciseDateFormatter(locator)
            axs[0].xaxis.set_major_locator(locator)
            axs[0].xaxis.set_major_formatter(formatter)        

            plt.show()
            
def apply_filter_handler(change):
    if (change.new == True):
        max_filter.disabled = False
        min_filter.disabled = False
    else:
        max_filter.disabled = True
        min_filter.disabled = True


In [10]:
# defining widgets
plot_fft_btn = w.Button(
    description='Plot FFT',
    disabled=False,
    button_style='success'
)

plot_container = w.Output()

check_filter = w.Checkbox(
    value=False,
    description='Apply filter',
    disabled=False,
    indent=False
)

min_filter = w.FloatText(
    description='Lower limit:',
    disabled=True
)

max_filter = w.FloatText(
    description='Upper limit:',
    disabled=True
)

dropdown_pv = w.Dropdown(options=data.columns.values)

# defining event calls
plot_fft_btn.on_click(plot_fft_static)
check_filter.observe(apply_filter_handler, names='value')

# defining layout elements

input_container = w.VBox([w.Label(value='Select one PV to analyze'), dropdown_pv, check_filter, min_filter, max_filter, plot_fft_btn], layout={'border': '1px solid white', 'padding': '10px', 'display': 'flex', 'align_items': 'center'})
output_container = plot_container

container = w.VBox([input_container, output_container], layout={'display': 'flex', 'align_items': 'center'})

# displaying widgets
display(container)



